# Group E S2 - Ukraine War -Twitter Promotion from raw to std layer

# Sections
* [1. Setup](#1)
  * [1.1 Start Hadoop](#1.1)  
  * [1.2 Search for Spark Installation](#1.2)
  * [1.3 Create SparkSession](#1.3)
* [2. Lab](#2)
  * [2.1 Check Files](#2.1)
  * [2.2 Read raw DataFrame](#2.3)
  * [2.3 Transform raw DataFrame](#2.3)
  * [2.4 Write DataFrame to std](#2.4)
  * [2.5 Code improvements](#2.5)

<a id='1'></a>
## 1. Setup

Since we are going to process data stored from HDFS let's start the service

<a id='1.1'></a>
### 1.1 Start Hadoop

Start Hadoop
Open a terminal and execute
```sh
hadoop-start.sh
```

<a id='1.2'></a>
### 1.2 Search for Spark Installation 
This step is required just because we are working in the course environment.

In [1]:
import findspark
findspark.init()

I'm changing pandas max column width property to improve data displaying

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

<a id='1.3'></a>
### 1.3 Create SparkSession

By setting this environment variable we can include extra libraries in our Spark cluster

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/hive3/lib/hive-hcatalog-core-3.1.2.jar pyspark-shell'

The first thing always is to create the SparkSession

In [4]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder\
.appName("RAW to STD - DataFrames")\
.config("spark.sql.warehouse.dir","hdfs://localhost:9000/warehouse")\
.config("spark.sql.legacy.timeParserPolicy","LEGACY")\
.config("spark.sql.sources.partitionOverwriteMode","dynamic")\
.enableHiveSupport()\
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<a id='2'></a>
## 2. Lab

<a id='2.1'></a>
### 2.1 Check  Files

To promote the twitter files, we need to ingest them using nifi as we describe in our report.
The files in raw format will be located here:

http://localhost:50070/explorer.html#/datalake/raw/twitter/War/

<a id='2.2'></a>
### 2.2 Read Raw DataFrame

We can infer the schema of the underlying json files by setting this option during the reading operation.<br/>
This is not recommended in production workloads as is very expensive (Spark will scan all the files in order to determine all the columns)


In [5]:
tweets_raw = spark.read.option("inferSchema","true")\
                       .option("recursiveFileLookup", "true")\
                       .json("hdfs://localhost:9000/datalake/raw/twitter/War/")
                       
tweets_raw.limit(5).toPandas()

contributors coordinates                      created_at display_text_range  \
0         None        None  Sun Feb 27 12:08:48 +0000 2022               None   
1         None        None  Sun Feb 27 12:08:48 +0000 2022               None   
2         None        None  Sun Feb 27 12:08:48 +0000 2022               None   
3         None        None  Sun Feb 27 12:08:48 +0000 2022               None   
4         None        None  Sun Feb 27 12:08:48 +0000 2022               None   

                                                                                                    entities  \
0            ([], None, [], [], [(80653307, 80653307, [3, 14], Mikhail Khodorkovsky (English), mbk_center)])   
1                    ([], None, [], [], [(1027991081584144386, 1027991081584144386, [3, 11], nws, load_pm)])   
2                  ([], None, [], [], [(1466263472132423681, 1466263472132423681, [3, 11], Raebo, Raebo56)])   
3  ([], None, [], [], [(1034205948007645184, 1034205948007645184, [3, 19], David Laufman, DavidLaufmanLaw)])   
4  ([], None, [], [], [(1475905928511766532, 1475905928511766532, [3, 19], Alexandre 𓅨𓂋𓀀, PessimismeActif)])   

  extended_entities extended_tweet  favorite_count  favorited filter_level  \
0              None           None               0      False          low   
1              None           None               0      False          low   
2              None           None               0      False          low   
3              None           None               0      False          low   
4              None           None               0      False          low   

   ... reply_count  retweet_count retweeted  \
0  ...           0              0     False   
1  ...           0              0     False   
2  ...           0              0     False   
3  ...           0              0     False   
4  ...           0              0     False   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Setting up the schema during data reading.

In [6]:
schema="""
created_at string,
id_str string,
text string,

`user` struct<
            name:string,
            screen_name:string,
            location:string,
            description:string,
            followers_count:bigint,
            friends_count:bigint,
            listed_count:bigint,
            favourites_count:bigint,
            statuses_count:bigint,
            created_at:string
            >,
retweeted_status struct <
            quot_count:int,
            reply_count:int,
            retweet_count:int,
            favorite_count:int,
            user: 
            struct <
                id_str:string,
                name:string,
                screen_name:string,
                location:string,
                description:string,
                followers_count:bigint,
                friends_count:bigint,
                listed_count:bigint,
                favourites_count:bigint,
                statuses_count:bigint,
                created_at:string
            >
            >,
place struct<
            country:string,
            country_code:string,
            full_name:string,
            place_type:string,
            url:string
            >,
quote_count bigint,
reply_count bigint,
retweet_count bigint,
favorite_count bigint,
entities struct<
            user_mentions:array<struct<screen_name:string>>,
            hashtags:array<struct<text:string>>, 
            media:array<struct<expanded_url:string>>, 
            urls:array<struct<expanded_url:string>>, 
            symbols:array<struct<text:string>>
            >,
favorited boolean,
retweeted boolean,
possibly_sensitive boolean,
filter_level string,
lang string
"""
tweets_raw = spark.read.schema(schema)\
                       .option("recursiveFileLookup", "true")\
                       .json("hdfs://localhost:9000/datalake/raw/twitter/War/")
                       
tweets_raw.limit(5).toPandas()


,created_at,id_str,text,user,retweeted_status,place,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,possibly_sensitive,filter_level,lang
0,Sun Feb 27 12:08:48 +0000 2022,1497906560814796801,"RT @mbk_center: Putin's invasion of Ukraine🇺🇦 has sparked a host of anti-war demonstrations in cities around the world, including Russia 🇷🇺…","(Abęfę🤴, abefe007, Ogun, Nigeria, Graphics Designer | Travel and Tour consultant | Man United Fan⚽ | Music Lover | Movie lover.... I always Follow Back✌️, 188, 1091, 0, 4075, 3347, Sun Apr 03 21:06:31 +0000 2016)","(None, 16, 107, 588, (80653307, Mikhail Khodorkovsky (English), mbk_center, London, Mikhail Khodorkovsky @mich261213 Twitter Account in English, 28401, 1664, 702, 17519, 35576, Wed Oct 07 19:03:44 +0000 2009))",None,0,0,0,0,"([(mbk_center,)], [], None, [], [])",False,False,None,low,en
1,Sun Feb 27 12:08:48 +0000 2022,1497906561049583620,"RT @load_pm: 💪Зараз у Хмельницькому жінки та діти плетуть маскувальні сітки, щоб допомогти нашим героям. Всі українці, від малого до велико…","(Piotr, Grado_Wwa, None, None, 1, 14, 0, 37, 7, Mon Nov 28 23:57:34 +0000 2016)","(None, 0, 1, 12, (1027991081584144386, nws, load_pm, UA Кременчук, 2022 Russian invasion of Ukraine, 1652, 14, 21, 108, 395, Fri Aug 10 18:52:27 +0000 2018))",None,0,0,0,0,"([(load_pm,)], [], None, [], [])",False,False,None,low,uk
2,Sun Feb 27 12:08:48 +0000 2022,1497906561058062340,RT @Raebo56: Don’t let the Russia invasion of Ukraine allow you to be distracted from the fact that \nJustin Trudeau and Jagmeet Singh just…,"(Sheila J, Sheilaanne2191, Montreal, Married, one adult son and a beautiful dog named Lily. Love all creatures except some humans I know:) Very into Canadian, British and U.S. politics., 525, 1225, 4, 53826, 31286, Sun Feb 11 18:57:47 +0000 2018)","(None, 17, 184, 440, (1466263472132423681, Raebo, Raebo56, Alberta, Canada, Albertan, Secessionist, Political Activist, Conservative, Business Owner, Patent Holder, Inventor, Jack Russells, Aviation, Cosmetic Chemistry, Photography, 562, 594, 1, 3086, 7581, Thu Dec 02 04:31:03 +0000 2021))",None,0,0,0,0,"([(Raebo56,)], [], None, [], [])",False,False,None,low,en
3,Sun Feb 27 12:08:48 +0000 2022,1497906561074741250,RT @DavidLaufmanLaw: Correction: It wasn’t “the Trump Administration” that required RT’s U.S. affiliate to register under the Foreign Agent…,"(TerryDean, TerryDe53425222, New York, USA, expat returned after little over 20 years. Trying to understand what’s going on?!?, 49, 120, 0, 47309, 6838, Tue Jun 02 20:27:08 +0000 2020)","(None, 4, 111, 433, (1034205948007645184, David Laufman, DavidLaufmanLaw, Washington, DC, Representation in gov’t investigations and national security matters. Former Chief of DOJ Counterintelligence Section. Stalwart Houston Astros fan. Views my own, 29303, 1286, 259, 3553, 991, Mon Aug 27 22:28:07 +0000 2018))",None,0,0,0,0,"([(DavidLaufmanLaw,)], [], None, [], [])",False,False,None,low,en
4,Sun Feb 27 12:08:48 +0000 2022,1497906560932134913,"RT @PessimismeActif: Oui, l’Ukraine est plus chère et proche à nos cœurs que l’Afghanistan ou le Togo.\n\nEt on s’excusera de rien du tout.","(lemmerdeuse 🇫🇷🇮🇱Z, lemmerdeuse24, Aquitaine, France, TOUS A VOS TEE SHIRT pour \nLIBERTE - EGALITE - FRATERNITE -\nRUES DE FRANCES LAÏQUES -\nLA RELIGION A LA MAISON -\nPOUR UNE FRANCE EN PAIX et SANS DISCREMINATION, 1280, 1619, 14, 18668, 22247, Sat Oct 12 12:13:06 +0000 2013)","(None, 14, 33, 154, (1475905928511766532, Alexandre 𓅨𓂋𓀀, PessimismeActif, Paris, France, Chats, Géopolitique & Pessimisme | moi/je, 1125, 307, 1, 3613, 2438, Tue Dec 28 19:06:21 +0000 2021))",None,0,0,0,0,"([(PessimismeActif,)], [], None, [], [])",False,False,None,low,fr


<a id='2.3'></a>
### 2.3 Transform Raw DataFrame
We transform the column created_at to its proper timestamp data type because is a string type and also we created some derive columns (dt, year, hour and day-hour) to store data information.<br/>

In [7]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *

In [8]:
tweets_std = tweets_raw\
             .withColumn("created_at",F.to_timestamp(F.col("created_at"),"EEE MMM dd HH:mm:ss ZZZZZ yyyy"))\
             .withColumn("year",F.year("created_at"))\
             .withColumn("dt",F.to_date("created_at"))\
             .withColumn("hour",F.hour("created_at"))\
             .withColumn("day-hour",concat_ws("-",F.to_date("created_at"),F.hour("created_at")))
                
tweets_std.limit(50).toPandas()

,created_at,id_str,text,user,retweeted_status,place,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,possibly_sensitive,filter_level,lang,year,dt,hour,day-hour
0,2022-02-27 13:08:48,1497906560814796801,"RT @mbk_center: Putin's invasion of Ukraine🇺🇦 has sparked a host of anti-war demonstrations in cities around the world, including Russia 🇷🇺…","(Abęfę🤴, abefe007, Ogun, Nigeria, Graphics Designer | Travel and Tour consultant | Man United Fan⚽ | Music Lover | Movie lover.... I always Follow Back✌️, 188, 1091, 0, 4075, 3347, Sun Apr 03 21:06:31 +0000 2016)","(None, 16, 107, 588, (80653307, Mikhail Khodorkovsky (English), mbk_center, London, Mikhail Khodorkovsky @mich261213 Twitter Account in English, 28401, 1664, 702, 17519, 35576, Wed Oct 07 19:03:44 +0000 2009))",None,0,0,0,0,"([(mbk_center,)], [], None, [], [])",False,False,None,low,en,2022,2022-02-27,13,2022-02-27-13
1,2022-02-27 13:08:48,1497906561049583620,"RT @load_pm: 💪Зараз у Хмельницькому жінки та діти плетуть маскувальні сітки, щоб допомогти нашим героям. Всі українці, від малого до велико…","(Piotr, Grado_Wwa, None, None, 1, 14, 0, 37, 7, Mon Nov 28 23:57:34 +0000 2016)","(None, 0, 1, 12, (1027991081584144386, nws, load_pm, UA Кременчук, 2022 Russian invasion of Ukraine, 1652, 14, 21, 108, 395, Fri Aug 10 18:52:27 +0000 2018))",None,0,0,0,0,"([(load_pm,)], [], None, [], [])",False,False,None,low,uk,2022,2022-02-27,13,2022-02-27-13
2,2022-02-27 13:08:48,1497906561058062340,RT @Raebo56: Don’t let the Russia invasion of Ukraine allow you to be distracted from the fact that \nJustin Trudeau and Jagmeet Singh just…,"(Sheila J, Sheilaanne2191, Montreal, Married, one adult son and a beautiful dog named Lily. Love all creatures except some humans I know:) Very into Canadian, British and U.S. politics., 525, 1225, 4, 53826, 31286, Sun Feb 11 18:57:47 +0000 2018)","(None, 17, 184, 440, (1466263472132423681, Raebo, Raebo56, Alberta, Canada, Albertan, Secessionist, Political Activist, Conservative, Business Owner, Patent Holder, Inventor, Jack Russells, Aviation, Cosmetic Chemistry, Photography, 562, 594, 1, 3086, 7581, Thu Dec 02 04:31:03 +0000 2021))",None,0,0,0,0,"([(Raebo56,)], [], None, [], [])",False,False,None,low,en,2022,2022-02-27,13,2022-02-27-13
3,2022-02-27 13:08:48,1497906561074741250,RT @DavidLaufmanLaw: Correction: It wasn’t “the Trump Administration” that required RT’s U.S. affiliate to register under the Foreign Agent…,"(TerryDean, TerryDe53425222, New York, USA, expat returned after little over 20 years. Trying to understand what’s going on?!?, 49, 120, 0, 47309, 6838, Tue Jun 02 20:27:08 +0000 2020)","(None, 4, 111, 433, (1034205948007645184, David Laufman, DavidLaufmanLaw, Washington, DC, Representation in gov’t investigations and national security matters. Former Chief of DOJ Counterintelligence Section. Stalwart Houston Astros fan. Views my own, 29303, 1286, 259, 3553, 991, Mon Aug 27 22:28:07 +0000 2018))",None,0,0,0,0,"([(DavidLaufmanLaw,)], [], None, [], [])",False,False,None,low,en,2022,2022-02-27,13,2022-02-27-13
4,2022-02-27 13:08:48,1497906560932134913,"RT @PessimismeActif: Oui, l’Ukraine est plus chère et proche à nos cœurs que l’Afghanistan ou le Togo.\n\nEt on s’excusera de rien du tout.","(lemmerdeuse 🇫🇷🇮🇱Z, lemmerdeuse24, Aquitaine, France, TOUS A VOS TEE SHIRT pour \nLIBERTE - EGALITE - FRATERNITE -\nRUES DE FRANCES LAÏQUES -\nLA RELIGION A LA MAISON -\nPOUR UNE FRANCE EN PAIX et SANS DISCREMINATION, 1280, 1619, 14, 18668, 22247, Sat Oct 12 12:13:06 +0000 2013)","(None, 14, 33, 154, (1475905928511766532, Alexandre 𓅨𓂋𓀀, PessimismeActif, Paris, France, Chats, Géopolitique & Pessimisme | moi/je, 1125, 307, 1, 3613, 2438, Tue Dec 28 19:06:21 +0000 2021))",None,0,0,0,0,"([(PessimismeActif,)], [], None, [], [])",False,False,None,low,fr,2022,2022-02-27,13,2022-02-27-13
5,2022-02-27 13:08:48,1497906561125023745,"RT @thehill: Germany says its sending anti-tank weapons, stinger missiles to Ukraine https://t.co/YwIi7KJPPn ht

<a id='2.4'></a>
### 2.4 Write DataFrame to std

In this case since a single day of data would be comprised of multiple files (partitions) I'm going to regroup them in just one with the coalesce method).<br/>
Also I'm going to partition the data using the columns created in the previous step.

In [9]:
(tweets_std.coalesce(1)
          .write
          .partitionBy("year","dt","day-hour")
          .mode("overwrite")
          .parquet("hdfs://localhost:9000/datalake/std/twitter/War/"))

<a id='2.5'></a>
### 2.5 Code improvements

The previous code it will read all the files from raw and write them to std layer.<br/>
This is not a good idea since we will convert and save the same data over and over if we execute this code every day.<br/>
With time based datasets like this one is typical just to take one day of data to just propote previous data to std layer without having to replay the transformations of previous days.<br/>
We also adapt the code with retweeted_Status structure as the original columns for counting favorites and retweets did not get data, and after reviewing Twitter API we decided those columns were the right ones.

In [10]:
from pyspark.sql.functions import *             

def promote_raw2std(day):
    schema="""
        created_at string,
        id_str string,
        text string,

        `user` struct<
                    name:string,
                    screen_name:string,
                    location:string,
                    description:string,
                    followers_count:bigint,
                    friends_count:bigint,
                    listed_count:bigint,
                    favourites_count:bigint,
                    statuses_count:bigint,
                    created_at:string
                    >,
        retweeted_status struct <
                    quot_count:int,
                    reply_count:int,
                    retweet_count:int,
                    favorite_count:int,
                    user: 
                    struct <
                        id_str:string,
                        name:string,
                        screen_name:string,
                        location:string,
                        description:string,
                        followers_count:bigint,
                        friends_count:bigint,
                        listed_count:bigint,
                        favourites_count:bigint,
                        statuses_count:bigint,
                        created_at:string
                    >
                    >,
        place struct<
                    country:string,
                    country_code:string,
                    full_name:string,
                    place_type:string,
                    url:string
                    >,
        quote_count bigint,
        reply_count bigint,
        retweet_count bigint,
        favorite_count bigint,
        entities struct<
                    user_mentions:array<struct<screen_name:string>>,
                    hashtags:array<struct<text:string>>, 
                    media:array<struct<expanded_url:string>>, 
                    urls:array<struct<expanded_url:string>>, 
                    symbols:array<struct<text:string>>
                    >,
        favorited boolean,
        retweeted boolean,
        possibly_sensitive boolean,
        filter_level string,
        lang string,
        hour int,
        year int,
        mix string
        """
    raw_location = f"hdfs://localhost:9000/datalake/raw/twitter/War/{day}/"
    (spark.read.schema(schema)
                       .option("recursiveFileLookup", "true")
                       .json(raw_location)
                       .withColumn("created_at",to_timestamp(col("created_at"),"EEE MMM dd HH:mm:ss ZZZZZ yyyy"))
                       .withColumn("year",year("created_at"))
                       .withColumn("dt",to_date("created_at"))
                       .withColumn("day-hour",concat_ws("-",F.to_date("created_at"),F.hour("created_at")))
                       .coalesce(1)
                       .write
                       .partitionBy("year","dt","day-hour")
                       .mode("overwrite")
                       .parquet("hdfs://localhost:9000/datalake/std/twitter/War/"))

In [11]:
tweets_std.printSchema()

root
 |-- created_at: timestamp (nullable = true)
 |-- id_str: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- screen_name: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- listed_count: long (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- statuses_count: long (nullable = true)
 |    |-- created_at: string (nullable = true)
 |-- retweeted_status: struct (nullable = true)
 |    |-- quot_count: integer (nullable = true)
 |    |-- reply_count: integer (nullable = true)
 |    |-- retweet_count: integer (nullable = true)
 |    |-- favorite_count: integer (nullable = true)
 |    |-- user: struct (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- name: string (

Now we can promote the data per year, in this case, 2022.<br/>


In [12]:
# Change this date according to your data in HDFS

#promote_raw2std("2021/12/06")

#promote_raw2std("2021/12")

promote_raw2std("2022")